In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/train_qa.csv")

print("File loaded successfully!")


File loaded successfully!


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

EMBEDDING_DIM = 64 


stories = df['story'].values.tolist()
questions = df['question'].values.tolist()

answers = df['answer'].values.tolist()

tokenizer = Tokenizer(filters=[], lower=True)
tokenizer.fit_on_texts(stories + questions)

VOCAB_SIZE = len(tokenizer.word_index) + 1
print(f"Total Vocabulary Size: {VOCAB_SIZE}")


story_sequences = tokenizer.texts_to_sequences(stories)
question_sequences = tokenizer.texts_to_sequences(questions)


MAX_STORY_LEN = max(len(s) for s in story_sequences)
MAX_QUESTION_LEN = max(len(q) for q in question_sequences)

print(f"Max Story Length: {MAX_STORY_LEN}")
print(f"Max Question Length: {MAX_QUESTION_LEN}")


padded_stories = pad_sequences(story_sequences, maxlen=MAX_STORY_LEN, padding='post')
padded_questions = pad_sequences(question_sequences, maxlen=MAX_QUESTION_LEN, padding='post')


target_answers = np.array([1 if a.lower() == 'yes' else 0 for a in answers])


train_stories, test_stories, train_questions, test_questions, y_train, y_test = train_test_split(
    padded_stories, padded_questions, target_answers, test_size=0.2, random_state=42
)

print("\n--- Final Data Shapes ---")
print(f"Training Stories shape: {train_stories.shape}")
print(f"Training Questions shape: {train_questions.shape}")
print(f"Training Answers shape: {y_train.shape}")

Total Vocabulary Size: 36
Max Story Length: 156
Max Question Length: 6

--- Final Data Shapes ---
Training Stories shape: (8000, 156)
Training Questions shape: (8000, 6)
Training Answers shape: (8000,)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping


VOCAB_SIZE = 36         
MAX_STORY_LEN = 156     
MAX_QUESTION_LEN = 6    
EMBEDDING_DIM = 64      


story_input = Input(shape=(MAX_STORY_LEN,), dtype='int32', name='story_input')


story_embedding = Embedding(
    input_dim=VOCAB_SIZE,
    output_dim=EMBEDDING_DIM,
    mask_zero=True 
)(story_input)


story_lstm = LSTM(EMBEDDING_DIM)(story_embedding)



question_input = Input(shape=(MAX_QUESTION_LEN,), dtype='int32', name='question_input')


question_embedding = Embedding(
    input_dim=VOCAB_SIZE,
    output_dim=EMBEDDING_DIM,
    mask_zero=True
)(question_input)


question_lstm = LSTM(EMBEDDING_DIM)(question_embedding)



merged = concatenate([story_lstm, question_lstm])


dense1 = Dense(EMBEDDING_DIM, activation='relu')(merged)
dense1 = Dropout(0.5)(dense1) 


output_tensor = Dense(1, activation='sigmoid')(dense1)


model = Model(inputs=[story_input, question_input], outputs=output_tensor)


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Keras Dual-Encoder LSTM Model Summary:")
model.summary()

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

print("\nStarting Model Training...")

history = model.fit(
    x=[train_stories, train_questions], 
    y=y_train,
    batch_size=64,
    epochs=20, 
    validation_data=([test_stories, test_questions], y_test),
    callbacks=[early_stopping]
)

print("\n--- Training Complete ---")

Keras Dual-Encoder LSTM Model Summary:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ story_input         │ (None, 156)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ question_input      │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 156, 64)   │      2,304 │ story_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 156)       │          0 │ story_input[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 6, 64)     │      2,304 │ question_input[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 6)         │          0 │ question_input[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     33,024 │ embedding[0][0],  │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64)        │     33,024 │ embedding_1[0][0… │
│                     │                   │            │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128)       │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      8,256 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         65 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 78,977 (308.50 KB)

 Trainable params: 78,977 (308.50 KB)

 Non-trainable params: 0 (0.00 B)


Starting Model Training...
Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.4959 - loss: 0.6936 - val_accuracy: 0.5120 - val_loss: 0.6921
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.5814 - loss: 0.6772 - val_accuracy: 0.6470 - val_loss: 0.6371
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.6525 - loss: 0.6298 - val_accuracy: 0.6765 - val_loss: 0.6071
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6711 - loss: 0.6059 - val_accuracy: 0.7240 - val_loss: 0.5755
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6963 - loss: 0.5767 - val_accuracy: 0.7235 - val_loss: 0.5475
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7154 - loss: 0.5534 - val_accuracy: 0.7505 - val_loss: 0.5221
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7280 - loss: 0.5408 - val_accuracy: 0.7590 - val_loss: 0.5087
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.74

In [ ]:
.

def predict_story_answer(story: str, question: str):
    """
    Takes a raw story and question, preprocesses them, and returns a Yes/No answer.
    """
   
    story_seq = tokenizer.texts_to_sequences([story])
    question_seq = tokenizer.texts_to_sequences([question])

    
    padded_story = pad_sequences(story_seq, maxlen=MAX_STORY_LEN, padding='post')
    padded_question = pad_sequences(question_seq, maxlen=MAX_QUESTION_LEN, padding='post')

 
    prediction = model.predict([padded_story, padded_question], verbose=0)

    
    probability = prediction[0][0]

   
    if probability > 0.5:
        answer = "Yes"
    else:
        answer = "No"

    return answer, probability


test_story_1 = "Mary moved to the bathroom . Sandra journeyed to the bedroom . Daniel went back to the hallway . Mary went back to the bedroom ."
test_question_1 = "Is Mary in the bedroom ?"


test_story_2 = "Daniel went back to the hallway . Sandra went to the kitchen . Daniel went back to the bathroom . Daniel picked up the football there . Daniel went to the bedroom ."
test_question_2 = "Is Daniel in the hallway ?"


ans1, prob1 = predict_story_answer(test_story_1, test_question_1)
ans2, prob2 = predict_story_answer(test_story_2, test_question_2)


print(f"\n--- Test 1 ---")
print(f"Story: {test_story_1}")
print(f"Question: {test_question_1}")
print(f"Prediction: {ans1} (Confidence: {prob1:.4f})")
print(f"Expected Answer: Yes\n")

print(f"\n--- Test 2 ---")
print(f"Story: {test_story_2}")
print(f"Question: {test_question_2}")
print(f"Prediction: {ans2} (Confidence: {prob2:.4f})")
print(f"Expected Answer: No")


--- Test 1 ---
Story: Mary moved to the bathroom . Sandra journeyed to the bedroom . Daniel went back to the hallway . Mary went back to the bedroom .
Question: Is Mary in the bedroom ?
Prediction: Yes (Confidence: 0.8370)
Expected Answer: Yes


--- Test 2 ---
Story: Daniel went back to the hallway . Sandra went to the kitchen . Daniel went back to the bathroom . Daniel picked up the football there . Daniel went to the bedroom .
Question: Is Daniel in the hallway ?
Prediction: No (Confidence: 0.0187)
Expected Answer: No


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences


TEST_CSV_FILE_PATH = '/content/drive/MyDrive/test_qa.csv'

try:
    df_test = pd.read_csv(TEST_CSV_FILE_PATH)
except FileNotFoundError:
    print(f"Error: The test file '{TEST_CSV_FILE_PATH}' was not found. Please verify the path.")
    raise

print(f"Successfully loaded {len(df_test)} test samples.")


test_stories_raw = df_test['story'].values.tolist()
test_questions_raw = df_test['question'].values.tolist()
test_answers_raw = df_test['answer'].values.tolist()


test_stories_seq = tokenizer.texts_to_sequences(test_stories_raw)
test_questions_seq = tokenizer.texts_to_sequences(test_questions_raw)


padded_test_stories = pad_sequences(test_stories_seq, maxlen=MAX_STORY_LEN, padding='post')
padded_test_questions = pad_sequences(test_questions_seq, maxlen=MAX_QUESTION_LEN, padding='post')


y_test_final = np.array([1 if a.lower() == 'yes' else 0 for a in test_answers_raw])

print("\nTest data preprocessing complete.")
print(f"Test Stories shape: {padded_test_stories.shape}")
print(f"Test Answers shape: {y_test_final.shape}")


print("\n--- Running Final Model Evaluation on Test Set ---")

loss, accuracy = model.evaluate(
    x=[padded_test_stories, padded_test_questions],
    y=y_test_final,
    verbose=0
)

print(f"\n✅ Final Test Loss: {loss:.4f}")
print(f"✅ Final Test Accuracy: {accuracy:.4f}")

Successfully loaded 1000 test samples.

Test data preprocessing complete.
Test Stories shape: (1000, 156)
Test Answers shape: (1000,)

--- Running Final Model Evaluation on Test Set ---

✅ Final Test Loss: 0.3972
✅ Final Test Accuracy: 0.8160


In [ ]:
import pickle
from tensorflow.keras.models import save_model


SAVE_DIR = '/content/drive/MyDrive'


import os
os.makedirs(SAVE_DIR, exist_ok=True)

MODEL_PATH = os.path.join(SAVE_DIR, 'story_qa_lstm_model.h5')
TOKENIZER_PATH = os.path.join(SAVE_DIR, 'story_qa_tokenizer.pickle')

save_model(model, MODEL_PATH)
print(f" Model saved successfully to: {MODEL_PATH}")


with open(TOKENIZER_PATH, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f" Tokenizer saved successfully to: {TOKENIZER_PATH}")

✅ Model saved successfully to: /content/drive/MyDrive/story_qa_lstm_model.h5
✅ Tokenizer saved successfully to: /content/drive/MyDrive/story_qa_tokenizer.pickle


In [ ]:
import os
from tensorflow.keras.models import save_model


KERAS_MODEL_PATH = '/content/drive/MyDrive/story_qa_best_lstm_model.keras'


save_model(model, KERAS_MODEL_PATH)

print(f"✅ Model successfully re-saved in the modern .keras format to: {KERAS_MODEL_PATH}")

✅ Model successfully re-saved in the modern .keras format to: /content/drive/MyDrive/story_qa_best_lstm_model.keras


In [ ]:
import pickle
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os


KERAS_MODEL_PATH = '/content/drive/MyDrive/story_qa_best_lstm_model.keras'
TOKENIZER_PATH = "/content/drive/MyDrive/story_qa_tokenizer.pickle"

MAX_STORY_LEN = 156
MAX_QUESTION_LEN = 6

try:
    with open(TOKENIZER_PATH, 'rb') as handle:
        loaded_tokenizer = pickle.load(handle)
    print("✅ Tokenizer loaded successfully.")
except FileNotFoundError:
    print(f"Error: Tokenizer file not found at {TOKENIZER_PATH}")
    raise

try:
    loaded_model = load_model(KERAS_MODEL_PATH)
    print("✅ Model loaded successfully (Native .keras format).")
except Exception as e:
    print(f"Error loading model: {e}")
    raise



def predict_loaded_answer(story: str, question: str, tokenizer_obj, model_obj):
    story_seq = tokenizer_obj.texts_to_sequences([story])
    question_seq = tokenizer_obj.texts_to_sequences([question])

    padded_story = pad_sequences(story_seq, maxlen=MAX_STORY_LEN, padding='post')
    padded_question = pad_sequences(question_seq, maxlen=MAX_QUESTION_LEN, padding='post')

    
    prediction = model_obj.predict([padded_story, padded_question], verbose=0)
    probability = prediction[0][0]
    answer = "Yes" if probability > 0.5 else "No"

    return answer, probability

test_story = "Mary moved to the garden. Sandra journeyed to the kitchen. Mary went back to the bedroom."
test_question = "Is Mary in the kitchen?"


ans, prob = predict_loaded_answer(
    test_story,
    test_question,
    loaded_tokenizer,
    loaded_model
)

print(f"\n--- Test of Loaded Model ---")
print(f"Story: {test_story}")
print(f"Question: {test_question}")
print(f"Prediction: {ans} (Confidence: {prob:.4f})")
print(f"Expected Answer: No")

✅ Tokenizer loaded successfully.
✅ Model loaded successfully (Native .keras format).


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



--- Test of Loaded Model ---
Story: Mary moved to the garden. Sandra journeyed to the kitchen. Mary went back to the bedroom.
Question: Is Mary in the kitchen?
Prediction: No (Confidence: 0.2269)
Expected Answer: No
